In [2]:
import json
import warnings

import numpy as np
import pandas as pd
from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.models.popular import PopularModel

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

# Read data

In [3]:
interactions_df = pd.read_csv("../data/interactions.csv")
users = pd.read_csv("../data/users.csv")
items = pd.read_csv("../data/items.csv")

interactions_df.rename(columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight},
                       inplace=True)

test = False
if test:
    interactions = Interactions(interactions_df.sample(frac=0.01))
else:
    interactions = Interactions(interactions_df)

interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,"4,250.000000",72.000000
1,699317,1659,2021-05-29,"8,317.000000",100.000000
2,656683,7107,2021-05-09,10.000000,0.000000
3,864613,7638,2021-07-05,"14,483.000000",100.000000
4,964868,9506,2021-04-30,"6,725.000000",100.000000


# Training popular model

In [4]:
model = PopularModel()
df_train = interactions.df
dataset = Dataset.construct(
    interactions_df=df_train,
    user_features_df=None,
    item_features_df=None,
)
catalog = df_train[Columns.Item].unique()
model.fit(dataset)

# Recommendations prediction

In [5]:
test_users = np.unique(df_train[Columns.User])
recommendations = model.recommend(
    users=test_users,
    dataset=dataset,
    k=10,
    filter_viewed=False,
)
recommendations.head()

,user_id,item_id,score,rank
0,0,10440,"202,457.000000",1
1,0,15297,"193,123.000000",2
2,0,9728,"132,865.000000",3
3,0,13865,"122,119.000000",4
4,0,4151,"91,167.000000",5


### Prediction for cold users

In [7]:
recommendations_dict = recommendations.groupby("user_id")["item_id"].agg(list).to_dict()

default_value = model.recommend(
    users=[1],
    dataset=dataset,
    k=10,
    filter_viewed=False,
).item_id.to_list()

recommendations_dict["default"] = default_value

# Saving recommendation

In [8]:
with open("../model_weights/popular_model.json", "w") as f:
    json.dump(recommendations.groupby("user_id")["item_id"].agg(list).to_dict(), f)
